这个系列的跨度有点久了，在开始之前，做一个前情提要。这个系列我们选择的是中金2023年12月的一份研报，名为《在手之鸟，红利优选策略》。它是一个基本策略，通过这份研报的实现，我们可以了解到：

```` {note}
1. 获取众多基本面数据，并且我们会对部分数据的含义及编纂方法进行讲解
2. 月度调仓换股回测策略应该如何实现
3. 获得一个红利优选基础策略。通过适当改变参数，即可用以实战。
````


该策略要用到以下数据：

1. 行情数据。任何策略都默认需要它，至少会在计算远期收益时使用。 
2. 股息率，用来按股息率筛选个股，以及计算两年股息率均值因子。 
3. 分红数据。只有过去两年连续分红的公司才能入选。 
4. 审计意见。只有过去十年没有审计保留意见的公司才能入选。
5. 市值数据。只有市值大于50亿的公司才能入选。 
6. 净利润、营业收入和营业利润数据，用来计算净利润稳定性因子。
7. 股东数量变化 
8. 换手率。用来计算换手波动率。
9. pe_ttm，用来计算 ep 因子。
10. 经营现金流数据，用来计算经营现金流资产比因子。
11. 资产总计数据，与10一起，用来计算经营现金流资产比因子。 
12. 盈余公积金数据。与11一起，用来计算留存收益资产比因子。

In [1]:
import sys

sys.path.append(str(Path(".")))

from helper import (
    ParquetUnifiedStorage,
    dividend_yield_screen,
    fetch_bars,
    fetch_dv_ttm,
)
from moonshot import Moonshot


我们已经获得了第1~2步的数据，并实现了按股息率进行票池筛选。回测表明，仅仅是通过股息率因子，我们就可以获得一定的年化超额，和更好的夏普比。

在这一期，我们将探讨如何把分红数据也纳入进来。研报要求，只有过去两年连续分红的公司才能入选票池。这是一个看似简单，但实现上有一点点复杂度的需求。

## 获取分红数据

在上一篇中，我们已经获取了股息率数据，但要实现『连续两年分红』这个条件，为精确起见，我们不能把『连续两年股息率大于零』来当成『连续两年分红』，而是要直接获取分红原始记录。

```` {hint}
通过 daily_basic 获得的股息率是一个按过去12个月滚动计算的股息率。根据它的计算方式，就可能存在这样的情况，比如2023年12月进行了分红，2024年全年没有分红，但是直到2024年11月，股息率都会一直大于零。如此以来，在2025年2月，当我们问道，该股是否连续两年分红时，就会得到一个错误结果。在这方面，分红记录可以略微精确一点。
````


在 tushare 中，我们要通过 dividend 接口来获取分红数据。我们的回测发生在2018年到2023年之间，我们再一次遇到如何在 tushare 中，获取这么长跨度的数据的问题。根据我们之前的讨论，我们应该选择一次查询可以获得最多数据的接口（参数）。

该接口签名如下：

```{code-block}python
def dividend(ts_code: str|None = None, ann_date: str|None = None, record_date: str|None = None, ex_date: str|None = None, imp_ann_date: str|None = None):
    pass
```


但是，如果按这些参数来进行查询，每次返回的数据量会很小，导致获取数据时间过长。这里我们还发现了一个隐藏参数，大家可以根据自己的情况来决定是否采用。这个参数就是 end_date。我们把使用各个参数进行查询所能得到的记录数比较一下：

In [142]:
pro = ts.pro_api()

df_ann = pro.dividend(ann_date="20250419")
print("by ann_date", len(df_ann))

df_end = pro.dividend(end_date="20241231", offse=0, limit=6000)
print("by end_date", len(df_end))

df_ex = pro.dividend(ex_date="20250419")
print("by ex_date", len(df_ex))

df_record = pro.dividend(record_date="20250419")
print("by record_date", len(df_record))

df_imp = pro.dividend(imp_ann_date="20250419")
print("by imp_ann_date", len(df_imp))


2025-09-07 19:19:43.761 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


by ann_date 647
by end_date 2000
by ex_date 0
by record_date 0
by imp_ann_date 4


可以看到，使用 end_date 参数，可以获得的数据远远超过其它参数；但是，它的limit 并不是我们常见的6000，而是只能返回2000。这些行为上的不一致，是我们要注意的。

由于这里的 limit 只有2000，而现在 A 股有5000多支个股，所以，我们在通过 end_date获取数据时，还必须通过 offset/limit 多次调用，才能取全一天的数据。

下面的代码演示了如何取区间[start, end]之间的数据：

In [146]:
import time

def fetch_dividend(start: datetime.date, end: datetime.date):
    dates = pd.bdate_range(start, end)
    dfs = []
    limit = 2000
    for dt in dates:
        # 对每一个交易日，都可能有超过 limit 条记录
        for offset in range(0, 99):
            str_date = dt.strftime("%Y%m%d")
            df = pro.dividend(end_date=str_date, offset=offset * limit)
            dfs.append(df)
            if len(df) < 2000:
                break

    # 如果取太快，会导致 tushare 拒绝访问
    time.sleep(0.25)
    data = pd.concat(dfs)
    data["date"] = pd.to_datetime(data["ann_date"]).dt.date
    return data.rename(columns={"ts_code": "asset"})


在最后，我们对数据进行了一些处理，使得返回的数据包含 asset, date 这两列，以便我们能像其它数据一样，自动化地利用缓存。

现在，我们就用之前开发的缓存来保存这些数据：

In [148]:
# path = "/tmp/dividend.parquet"
store = ParquetUnifiedStorage(store_path=path)

for yr in (2023, ):
    start = datetime.date(yr, 1, 1)
    end = datetime.date(yr, 12, 31)

    data = fetch_dividend(start, end)
    # store.append_data(data[data["div_proc"] == "实施"])

data


2025-09-07 19:51:04.198 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


,asset,end_date,ann_date,div_proc,stk_div,stk_bo_rate,stk_co_rate,cash_div,cash_div_tax,record_date,ex_date,pay_date,div_listdate,imp_ann_date,date
0,002086.SZ,20230316,20230316,预案,1.586538,None,1.586538,0.000,0.000,None,None,None,None,None,2023-03-16
1,002086.SZ,20230316,20230316,股东大会通过,1.586569,None,1.586569,0.000,0.000,None,None,None,None,None,2023-03-16
2,002086.SZ,20230316,20230316,股东大会通过,1.590000,None,1.590000,0.000,0.000,None,None,None,None,None,2023-03-16
3,002086.SZ,20230316,20230316,实施,1.590000,None,1.590000,0.000,0.000,20231228,20231229,None,20231229,20231223,2023-03-16
4,002086.SZ,20230316,20230316,实施,1.590000,None,1.590000,0.000,0.000,20231228,20231229,None,20231229,20231223,2023-03-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,002508.SZ,20231214,20231214,实施,0.000000,None,NaN,0.500,0.500,20240109,20240110,20240110,None,20240103,2023-12-14
0,002969.SZ,20231226,20231226,预案,0.000000,None,NaN,0.000,0.230,None,None,None,None,None,2023-12-26
1,002969.SZ,20231226,20231226,股东大会通过,0.000000,None,NaN,0.000,0.230,None,None,None,None,None,2023-12-26
2,002969.SZ,20231226,20231226,实施,0.000000,None,NaN,0.230,0.230,20240123,20240124,20240124,None,20240118,2023-12-26


In [149]:
data[data.asset == "000001.SZ"]

,asset,end_date,ann_date,div_proc,stk_div,stk_bo_rate,stk_co_rate,cash_div,cash_div_tax,record_date,ex_date,pay_date,div_listdate,imp_ann_date,date
1592,000001.SZ,20230630,20230824,预案,0.0,None,NaN,0.0,0.0,None,None,None,None,None,2023-08-24


In [150]:
pro = ts.pro_api()
pro.dividend(ts_code="000001.SZ")


2025-09-07 19:56:24.128 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


,ts_code,end_date,ann_date,div_proc,stk_div,stk_bo_rate,stk_co_rate,cash_div,cash_div_tax,record_date,ex_date,pay_date,div_listdate,imp_ann_date
0,000001.SZ,20250630,20250823,预案,0.00,NaN,NaN,0.00000,0.2360,None,None,None,None,None
1,000001.SZ,20241231,20250315,预案,0.00,NaN,NaN,0.00000,0.3620,None,None,None,None,None
2,000001.SZ,20241231,20250315,股东大会通过,0.00,NaN,NaN,0.00000,0.3620,None,None,None,None,None
3,000001.SZ,20241231,20250315,实施,0.00,NaN,NaN,0.36200,0.3620,20250611,20250612,20250612,None,20250605
4,000001.SZ,20240630,20240816,预案,0.00,NaN,NaN,0.00000,0.2460,None,None,None,None,None
5,000001.SZ,20240630,20240816,实施,0.00,NaN,NaN,0.24600,0.2460,20241009,20241010,20241010,None,20240926
6,000001.SZ,20231231,20240315,预案,0.00,NaN,NaN,0.00000,0.7190,None,None,None,None,None
7,000001.SZ,20231231,20240315,股东大会通过,0.00,NaN,NaN,0.00000,0.7190,None,None,None,None,None
8,000001.SZ,20231231,20240315,实施,0.00,NaN,NaN,0.71900,0.7190,20240613,20240614,20240614,None,20240606
9,000001.SZ,20230630,20230824,预案,0.00,NaN,NaN,0.00000,0.0000,None,None,None,None,None


分段获取的原因是为了保证万一出错，我们也不会损失太多数据。

文档没有说明这些记录是如何编纂的。根据我们的分析，公司可能在一年内有多次分红（这样的公司太少了）；对每一次分红，它可能有多条记录，分别对应于预案、股东大会通过和实施三个阶段。

对我们本次需求来说，只要关注『预案』阶段的记录就好。因为预案一旦发布，相关炒作资金就会闻风而动，不会等到实施阶段。在预案中，我们又只需要关注 end_date, ann_date，这是计算是否有连续两年分红的关键。

但是，在示例代码，我们保存的是 div_proc 为『实施』阶段的数据。因为在大多数情况下，它包含了预案阶段的全部信息，同时又提供了实施阶段的一些额外信息，可以为今后使用。不过，我们要注意，在回测中，当我们把 ann_date 当成最新的时刻时，是不能去读 cash_div，record_date, ex_date 等信息的，此时它们还是『未来数据』

```` {warning}
tushare 的记录中给出了税后分红(cash_div)，但文档中并没有明确指出它的计算方法，欢迎讨论！按相关法规，企业与个人股东的分红税率不一样，个人股东持有时间长短不一样，分红税率也不一样。因此，理论上讲，每一个 cash_div_tax，都应该对应多个 cash_div -- 看谁来读它。
````


要如何判断某只股票是否连续分红呢？我们需要通过 end_date 提取出会计年度，并且将 ann_date 转换为年/月的格式。由于我们是在月末才进行调仓换股，假设现在是2020年6月30日，如果此时存在会计年度为2018， 2019年的两条以上记录，则可以认为该股连续两年分红了。而在回测中，我们还要加一条限制，才能防止使用未来数据。这条限制是，ann_date 必须小于等于 2020-06-30，即在回测时，已经可以拿到这两条数据了。

```` {warning}
做基本面回测的难度主要在于数据。在现实中，在2020年6月30日这一天，如果分红实施方案是在6月30日公布的，理论上你应该可以在当天晚上就得到数据，并且用它来决定投资策略。但是，这取决于你使用的数据源。对所有人都公开所得的数据，也不一定对你的计算机程序可得。如果你在实盘中使用的数据源处理起来没有那么快，那么，你的回测结果仍然无法用以实盘。
````


逻辑很简单。复杂性在于，如何高效地为每一个月生成 flag（以表明在该月，该股是否连续两年分红）。这里我们将使用以下技巧：

1. 通过月度和股票代码创建一个笛卡尔积，作为每支股票、每月 flag 的索引。这是最终我们所求结果的索引
2. 通过pivot_table 及聚合函数，快速生成个股每年分红表，用来向量化计算是否存在连续两年分红
3. 由表2和表1，把连续两年分红标记计算到月


## 预处理和生成索引

我们需要先将数据进行一点预处理，为每一条记录加上 fiscal_year 和 announce_ym 字段，并且生成一个空的 dataframe，用来存放处理后的数据。

In [30]:
cols = ["asset", "end_date", "ann_date"]
df = store.load_data(store.start, store.end)[cols]
df

2025-09-07 17:08:23.787 | INFO     | helper:load_data:371 - 从缓存加载数据: 20180201 到 20231130


,asset,end_date,ann_date
0,600733.SH,20180201,20180201
1,600828.SH,20180316,20180316
2,002192.SZ,20180424,20180424
3,601318.SH,20180427,20180427
4,000912.SZ,20180614,20180614
...,...,...,...
13038,600759.SH,20231019,20231019
13039,600519.SH,20231121,20231121
13040,000620.SZ,20231129,20231129
13041,603030.SH,20231129,20231129


## 数据预处理

In [135]:
df = store.load_data(store.start, store.end)[cols]

df["end_date"] = pd.to_datetime(df["end_date"])
df["ann_date"] = pd.to_datetime(df["ann_date"])
df["fiscal_year"] = df["end_date"].dt.year
df.drop("end_date", axis=1, inplace=True)
df["month"] = df["ann_date"].dt.to_period('M')

# 每一年都可能有多条记录，按这里的需求，我们只需要看最早宣布的记录
df = (df.sort_values(["fiscal_year", "asset", "month"])
      .groupby(["fiscal_year", "asset"])
      .first()
      .reset_index()
      .set_index(["asset", "month"])
)

df.xs("000001.SZ", level="asset")


2025-09-07 19:11:26.958 | INFO     | helper:load_data:371 - 从缓存加载数据: 20180201 到 20231130


,fiscal_year,ann_date
month,,
2019-03,2018,2019-03-07
2020-02,2019,2020-02-14
2021-02,2020,2021-02-02
2022-03,2021,2022-03-10


In [136]:
months = df.index.levels[1]
all_months = pd.period_range(
    start= months.min(), end=months.max(), freq="M"
)
all_assets = df.index.levels[0].unique()
index = pd.MultiIndex.from_product([all_assets, all_months,], names=["asset", "month"])
index

MultiIndex([('000001.SZ', '2018-02'),
            ('000001.SZ', '2018-03'),
            ('000001.SZ', '2018-04'),
            ('000001.SZ', '2018-05'),
            ('000001.SZ', '2018-06'),
            ('000001.SZ', '2018-07'),
            ('000001.SZ', '2018-08'),
            ('000001.SZ', '2018-09'),
            ('000001.SZ', '2018-10'),
            ('000001.SZ', '2018-11'),
            ...
            ('920819.BJ', '2023-02'),
            ('920819.BJ', '2023-03'),
            ('920819.BJ', '2023-04'),
            ('920819.BJ', '2023-05'),
            ('920819.BJ', '2023-06'),
            ('920819.BJ', '2023-07'),
            ('920819.BJ', '2023-08'),
            ('920819.BJ', '2023-09'),
            ('920819.BJ', '2023-10'),
            ('920819.BJ', '2023-11')],
           names=['asset', 'month'], length=307860)

下面创建结果集：

In [141]:
temp = pd.DataFrame(index=index).join(df, how = 'left')
result_df = temp.groupby(level = 0).ffill()
print(len(result_df))
result_df.xs("000001.SZ", level="asset")


307863


,fiscal_year,ann_date
month,,
2018-02,NaN,NaT
2018-03,NaN,NaT
2018-04,NaN,NaT
2018-05,NaN,NaT
2018-06,NaN,NaT
...,...,...
2023-07,2021.0,2022-03-10
2023-08,2021.0,2022-03-10
2023-09,2021.0,2022-03-10


In [138]:
mask = result_df.index.get_level_values("month").year - result_df["fiscal_year"] <= 1

result_df.loc[~mask, ["fiscal_year", "ann_date"]] = [np.nan, pd.NaT]
result_df.xs("000001.SZ", level="asset").iloc[-49:]

,fiscal_year,ann_date
month,,
2019-11,2018.0,2019-03-07
2019-12,2018.0,2019-03-07
2020-01,NaN,NaT
2020-02,2019.0,2020-02-14
2020-03,2019.0,2020-02-14
2020-04,2019.0,2020-02-14
2020-05,2019.0,2020-02-14
2020-06,2019.0,2020-02-14
2020-07,2019.0,2020-02-14


In [9]:
# 2. 计算连续两年分红的年份组合(当前年和前一年都有分红)
years = sorted(dividend_flags.columns)
consecutive_years = {}
for year in years[1:]:
    prev_year = year - 1
    if prev_year in years:
        consecutive_years[f'consec_{prev_year}_{year}'] = \
            (dividend_flags[prev_year] & dividend_flags[year]).astype(int)
        
consecutive_df = pd.DataFrame(consecutive_years, index=dividend_flags.index)
consecutive_df

,consec_2018_2019,consec_2019_2020,consec_2020_2021,consec_2021_2022,consec_2022_2023
asset,,,,,
000001.SZ,1,1,1,0,0
000002.SZ,1,1,1,0,0
000006.SZ,1,1,1,0,0
000008.SZ,1,0,0,0,0
000009.SZ,1,1,1,0,0
...,...,...,...,...,...
920445.BJ,0,0,3,1,0
920489.BJ,0,0,1,0,0
920682.BJ,0,0,0,0,0


In [154]:
store = ParquetUnifiedStorage(Path("~/workspace/data/rw/dividend.parquet").expanduser())
data = store.load_data(store.start, store.end)
data = data[data.asset == "000001.SZ"]
data[data.asset == "000001.SZ"]

2025-09-07 20:07:25.019 | INFO     | helper:load_data:371 - 从缓存加载数据: 20180201 到 20231130


,asset,end_date,ann_date,div_proc,stk_div,stk_bo_rate,stk_co_rate,cash_div,cash_div_tax,record_date,ex_date,pay_date,div_listdate,imp_ann_date,date,__index_level_0__
2664,000001.SZ,20181231,20190307,实施,0.0,NaN,NaN,0.145,0.145,20190625,20190626,20190626,None,20190620,20190307,1899
3345,000001.SZ,20191231,20200214,实施,0.0,NaN,NaN,0.218,0.218,20200527,20200528,20200528,None,20200522,20200214,74
7458,000001.SZ,20201231,20210202,实施,0.0,NaN,NaN,0.180,0.180,20210513,20210514,20210514,None,20210507,20210202,65
12314,000001.SZ,20211231,20220310,实施,0.0,NaN,NaN,0.228,0.228,20220721,20220722,20220722,None,20220715,20220310,748


In [140]:
payh = ts.pro_api().dividend(ts_code="000001.SZ")
payh = payh[payh.div_proc == "实施"]
payh

2025-09-07 19:13:20.694 | INFO     | tushare:_connect:29 - Connected to server tushare:5290


,ts_code,end_date,ann_date,div_proc,stk_div,stk_bo_rate,stk_co_rate,cash_div,cash_div_tax,record_date,ex_date,pay_date,div_listdate,imp_ann_date
3,000001.SZ,20241231,20250315,实施,0.00,NaN,NaN,0.36200,0.3620,20250611,20250612,20250612,None,20250605
5,000001.SZ,20240630,20240816,实施,0.00,NaN,NaN,0.24600,0.2460,20241009,20241010,20241010,None,20240926
8,000001.SZ,20231231,20240315,实施,0.00,NaN,NaN,0.71900,0.7190,20240613,20240614,20240614,None,20240606
12,000001.SZ,20221231,20230309,实施,0.00,NaN,NaN,0.28500,0.2850,20230613,20230614,20230614,None,20230607
16,000001.SZ,20211231,20220310,实施,0.00,NaN,NaN,0.22800,0.2280,20220721,20220722,20220722,None,20220715
20,000001.SZ,20201231,20210202,实施,0.00,NaN,NaN,0.18000,0.1800,20210513,20210514,20210514,None,20210507
24,000001.SZ,20191231,20200214,实施,0.00,NaN,NaN,0.21800,0.2180,20200527,20200528,20200528,None,20200522
28,000001.SZ,20181231,20190307,实施,0.00,NaN,NaN,0.14500,0.1450,20190625,20190626,20190626,None,20190620
30,000001.SZ,20171231,20180315,实施,0.00,NaN,NaN,0.13600,0.1360,20180711,20180712,20180712,None,20180706
31,000001.SZ,20161231,20170317,实施,0.00,NaN,NaN,0.15800,0.1580,20170720,20170721,20170721,None,20170717
